In [11]:
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

In [17]:
# Input metadata
img_width, img_height = 150, 150
ds_train = "./data/train"
ds_test = "./data/test"

In [19]:
# Divide the data
data_gen = ImageDataGenerator(rescale=1./255)
gen_train = data_gen.flow_from_directory(
    ds_train, 
    target_size=(img_width, img_height), 
    batch_size=16, 
    class_mode="binary"
)
gen_test = data_gen.flow_from_directory(
    ds_test, 
    target_size=(img_width, img_height), 
    batch_size=32, 
    class_mode="binary"
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [25]:
# Generate the CNN model
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  import sys
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # This is added back by InteractiveShellApp.init_path()


In [27]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [45]:
nb_epoch = 30
nb_train_samples = 2048
nb_validation_samples = 832

In [32]:
model.fit_generator(
        gen_train,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=gen_test,
        nb_val_samples=nb_validation_samples)

Epoch 1/30
  3/128 [..............................] - ETA: 5s - loss: 0.3972 - acc: 0.7917

/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=128, validation_steps=832, validation_data=<keras_pre..., epochs=30)`
  


128/128 [==============================] - 14s 108ms/step - loss: 0.4311 - acc: 0.8096 - val_loss: 0.4398 - val_acc: 0.7944
Epoch 2/30
128/128 [==============================] - 13s 104ms/step - loss: 0.4473 - acc: 0.7979 - val_loss: 0.4503 - val_acc: 0.7920
Epoch 3/30
128/128 [==============================] - 13s 103ms/step - loss: 0.4507 - acc: 0.8003 - val_loss: 0.5075 - val_acc: 0.7654
Epoch 4/30
128/128 [==============================] - 13s 102ms/step - loss: 0.4362 - acc: 0.8032 - val_loss: 0.4661 - val_acc: 0.8014
Epoch 5/30
128/128 [==============================] - 13s 102ms/step - loss: 0.4399 - acc: 0.8027 - val_loss: 0.5024 - val_acc: 0.7608
Epoch 6/30
128/128 [==============================] - 13s 104ms/step - loss: 0.4296 - acc: 0.8101 - val_loss: 0.4111 - val_acc: 0.8190
Epoch 7/30
128/128 [==============================] - 13s 103ms/step - loss: 0.4624 - acc: 0.7964 - val_loss: 0.4553 - val_acc: 0.8008
Epoch 8/30
128/128 [==============================] - 13s 103ms/st

In [37]:
model.save_weights("models/cnn_20_epochs.h5")

In [47]:
model.evaluate_generator(gen_train, nb_validation_samples)

[0.3325566362025431, 0.8618539663461539]